In [22]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from scipy.sparse import *
from scipy.sparse.linalg import svds
from gensim.models import Word2Vec
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors
from tqdm import tqdm

In [8]:
#플레이 리스트
train = pd.read_json('data/train.json')

#곡 정보
song_meta = pd.read_json('data/song_meta.json')

In [127]:
song_meta[song_meta['song_name'] == '만약에']

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
9975,[GN1801],20150814,"힐링음악, 명상음악이 필요할때 듣는 뉴에이지 피아노음악 Season 23",2334381,[859066],만약에,[GN1800],[뉴피아노],9975
68990,"[GN0105, GN0101]",20150327,나는 가수다 시즌3 9회 `OST`,2311398,[1099],만약에,[GN0100],[박정현],68990
135076,"[GN0104, GN0101]",20080516,더 스타쇼,380108,[246968],만약에,[GN0100],[김연아],135076
197000,[GN1801],20190116,인기가요 발라드 감성 피아노 연주곡 베스트,10242466,[2620673],만약에,[GN1800],[피아노 오딧세이 (Piano Odyssey)],197000
204302,"[GN0703, GN0701, GN0710]",20051020,"The Beautiful Soul, The Fourth Story",308989,[1165],만약에,[GN0700],[조항조],204302
211838,[GN0701],20130710,당신이 있기에,2195680,[438462],만약에,[GN0700],[박경존],211838
250827,"[GN0703, GN0708, GN0701]",20081120,Special Best,573559,[1165],만약에,[GN0700],[조항조],250827
253685,"[GN0805, GN0501, GN0502, GN0801, GN0509]",20161117,왼손편지,10016062,[44517],만약에,"[GN0500, GN0800]",[황영원],253685
293669,[GN0701],20070920,짱가 새바람,358675,[11949],만약에,[GN0700],[유지나],293669
342987,[GN0101],20111006,피아노로 듣는 발라드 인기가요 Best 45,2018449,[2727],만약에,[GN0100],[Various Artists],342987


In [125]:
song_meta

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4
...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987


In [126]:
train

,tags,id,plylst_title,songs,like_cnt,updt_date,song_newid
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,"[456704, 112732, 333158, 488440, 258853, 12127..."
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,"[375894, 587314, 431997, 104605, 338568, 21226..."
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,"[72132, 240434, 144495, 161861, 307991, 222934..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,"[342495, 169897, 469393, 250235, 383169, 87161..."
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,"[138494, 481095, 4399, 560460, 255877, 87448, ..."
...,...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,"[373452, 383795, 531854, 448795, 601027, 33646..."
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,"[279203, 187803, 464493, 208798, 287747, 20234..."
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,"[43717, 216378, 217755, 322592, 199796, 603802..."
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,"[463690, 528384, 298561, 362605, 529162, 26262..."


## 협업 필터링 희소행렬 만들기

In [9]:
all_songs = set(np.concatenate(train['songs']))
n_songs = len(all_songs)

song_to_newid  = dict()
newid_to_song = dict()

for i, t in enumerate(all_songs):
    song_to_newid[t] = i
    newid_to_song[i] = t

In [10]:
#기존 곡 id를 새 id로 바꾼 컬럼을 만든다.
train['song_newid'] = train['songs'].apply(lambda x: [song_to_newid[song] for song in x])
display(train['song_newid'])

0         [456704, 112732, 333158, 488440, 258853, 12127...
1         [375894, 587314, 431997, 104605, 338568, 21226...
2         [72132, 240434, 144495, 161861, 307991, 222934...
3         [342495, 169897, 469393, 250235, 383169, 87161...
4         [138494, 481095, 4399, 560460, 255877, 87448, ...
                                ...                        
115066    [373452, 383795, 531854, 448795, 601027, 33646...
115067    [279203, 187803, 464493, 208798, 287747, 20234...
115068    [43717, 216378, 217755, 322592, 199796, 603802...
115069    [463690, 528384, 298561, 362605, 529162, 26262...
115070    [22532, 396641, 281634, 78058, 117563, 124743,...
Name: song_newid, Length: 115071, dtype: object

In [11]:
row = np.repeat(range(len(train)), train['songs'].map(lambda x : len(x)))
col =  np.concatenate(train['song_newid'])
data = np.ones(col.shape[0])

In [13]:
ply_song_table = csr_matrix((data, (row,col)), shape= (len(train), n_songs))
song_ply_table = ply_song_table.T

In [14]:
song_ply_table

<615142x115071 sparse matrix of type '<class 'numpy.float64'>'
	with 5285871 stored elements in Compressed Sparse Column format>

In [16]:
#곡간 유사도 계산.
song_sim = cosine_similarity(song_ply_table, song_ply_table, dense_output = False)

---

## word2vec을 사용한 것과 비교 

In [41]:
# 노래 번호들을 스트링 형식으로
songs_list = [list(map(str, x)) for x in train['songs'].tolist()]

In [44]:
# # w2v 훈련
# w2v_model = Word2Vec(songs_list, min_count = 3, size = 100 , window = 200, sg = 1)
# #2~30분 소요

In [53]:
# 모델 저장.
# w2v_model.save('songid_w2v.model')

# 모델 불러오기
# w2v_model = Word2Vec.load('songid_w2v.model')

In [111]:
def get_15_songs(songid):
    '''
    return : 아이템기반 협업 필터링  / word2vec 사용 순위 DF
    '''
    top15_temp = [x[0] for x in w2v_model.wv.most_similar(str(songid), topn = 15)] 
    top15_w2v = list(map(int, top15_temp))
    
    top15_temp =  song_sim[song_to_newid[songid]].toarray().reshape(-1).argsort()[::-1][1:16]
    top15_cf = [newid_to_song[x] for x in top15_temp]
    
    a =  song_meta.loc[top15_cf, ['song_name','artist_name_basket']].reset_index(drop = True)
    b =  song_meta.loc[top15_w2v, ['song_name','artist_name_basket']].reset_index(drop = True)
    
    return pd.concat([a,b], axis = 1)

In [112]:
# 밤편지 - 아이유
get_15_songs(144663)
# 아이템기반 협업필터링 / word2vec 사용 순위

,song_name,artist_name_basket,song_name,artist_name_basket
0,팔레트 (Feat. G-DRAGON),[아이유],이런 엔딩,[아이유]
1,비도 오고 그래서 (Feat. 신용재),[헤이즈 (Heize)],이름에게,[아이유]
2,가을 아침,[아이유],비도 오고 그래서 (Feat. 신용재),[헤이즈 (Heize)]
3,사랑이 잘 (With 오혁),[아이유],사랑이 잘 (With 오혁),[아이유]
4,이름에게,[아이유],all of my life,[박원]
5,이런 엔딩,[아이유],팔레트 (Feat. G-DRAGON),[아이유]
6,솔직하게 말해서 나,[김나영],가을 아침,[아이유]
7,좋니,[윤종신],노력,[박원]
8,가을 타나 봐,[바이브],좋니,[윤종신]
9,가을 안부,[먼데이 키즈 (Monday Kiz)],끝,[권진아]


In [113]:
#오아시스 - 브아걸
get_15_songs(581535)
# 아이템기반 협업필터링 / word2vec 사용 순위

,song_name,artist_name_basket,song_name,artist_name_basket
0,이별 준비 (Remix),[임성은],디스코왕,[KYT (코요태)]
1,Adios,[화란],Summer Dance (Trance Mix),[이정현]
2,서투른 사랑 (Mojo House Re-mix),[채연],여름아부탁해,[비욘드]
3,동그라미,[KYT (코요태)],바다로 가자,[쟈크와 래미]
4,상처,[파티타임],여름 이야기,[DJ DOC]
5,My Love Ⅱ,[Honeybee],고백,[Funny]
6,이별이 안부를 묻다... (Narration By 차태현),[신지 (SHINJI)],Festival,[천상지희 더 그레이스 (The Grace)]
7,나야 (N.A.Y.A),[소냐],바다,[플라워]
8,애련,[KYT (코요태)],해변으로가요,[비쥬]
9,회심가 (Remix),[태사자],Hi Ya Ya 여름날 (TV-Mix Ver.),[동방신기 (TVXQ!)]


> 여름 노래를 기대.  오아시스의 경우 word2vec에서 더 의도한 대로 나온 것 같다.
- 다른 여름노래로 해보자

In [114]:
#태양은 가득히 - mc the max
get_15_songs(368863)
# 아이템기반 협업필터링 / word2vec 사용 순위

,song_name,artist_name_basket,song_name,artist_name_basket
0,해변으로 가요,[레인보우],태양은 가득히 (2007 New Ver.),[엠씨더맥스 (M.C the MAX)]
1,Run To The Sky (Cooler Remix) (Inst.),[엠씨더맥스 (M.C the MAX)],태양은 가득히 (2007 New Ver.),[엠씨더맥스 (M.C the MAX)]
2,황진이 (Remix),[박상철],Run To The Sky (Cooler Remix),[엠씨더맥스 (M.C the MAX)]
3,난 그래 (2016 Live Ver.),[엠씨더맥스 (M.C the MAX)],Summer Couple (태양은 가득히),[문차일드]
4,사랑합니다 (Inst.),[엠씨더맥스 (M.C the MAX)],굿바이 세러머니,[바나나 보트]
5,가슴에 내린다 (Chinese Ver.),[이수 (엠씨더맥스)],여름아부탁해,[비욘드]
6,Intro,[엠씨더맥스 (M.C the MAX)],쿵따리샤바라 (클론),[아이씨사이다]
7,흥보가 기가 막혀,[사계절],바다의공주 (Club Ver.),[LPG Loves 래피]
8,My way (Inst.),[이수 (엠씨더맥스)],태양은 가득히 (2007 New Ver.),[엠씨더맥스 (M.C the MAX)]
9,Sixth Sense,[엠씨더맥스 (M.C the MAX)],Summer Day,[캔]


In [115]:
#여름아! 부탁해 - 인디고
get_15_songs(667123)
# 아이템기반 협업필터링 / word2vec 사용 순위

,song_name,artist_name_basket,song_name,artist_name_basket
0,파도,[유엔],여름안에서 (Original Ver.),[서연]
1,이여름 Summer,[쿨 (COOL)],파도,[유엔]
2,냉면,[명카드라이브],이여름 Summer,[쿨 (COOL)]
3,해변의 여인,[쿨 (COOL)],바다의 왕자,[박명수]
4,오아시스 (Feat. 이재훈),[브라운아이드걸스],여름안에서,[서연]
5,팥빙수,[윤종신],여름 이야기,[DJ DOC]
6,Summer Couple (태양은 가득히),[문차일드],Summer Dance,[이정현]
7,바다의 왕자,[박명수],해변의 여인,[쿨 (COOL)]
8,Summer Dance,[이정현],여름이야기 (Original Ver.),[DJ DOC]
9,Summer Vacation,[SMTOWN],여름이잖아요,[2BIC(투빅)]


In [118]:
#all i want for christmas is you - 머라이어 캐리
get_15_songs(78027)
# 아이템기반 협업필터링 / word2vec 사용 순위

,song_name,artist_name_basket,song_name,artist_name_basket
0,A Holly Jolly Christmas (Single Version),[Burl Ives],A Holly Jolly Christmas,[Burl Ives]
1,Adeste Fideles,[Helene Fischer],2000 Miles,[The Pretenders]
2,The Way Life Goes (Feat. Oh Wonder),[Lil Uzi Vert],When Christmas Comes Around,[Matt Terry]
3,Rockin` Around The Christmas Tree,[Brenda Lee],Rockin` Around The Christmas Tree,[Brenda Lee]
4,Silver Bells (Remastered),[Pat Boone],Happy Xmas (War Is Over) (Feat. The Harlem Com...,"[John Lennon, The Plastic Ono Band]"
5,The First Noel (Remastered),"[Pat Boone, Lew Douglas]",Merry Xmas Everybody,[Slade]
6,Little Drummer Boy,[Boney M],Merry Christmas Everyone,[Shakin` Stevens]
7,It Came Upon A Midnight Clear (Remastered),[Ella Fitzgerald],Fairytale Of New York (Feat. Kirsty Maccoll),[The Pogues]
8,Codeine Dreaming (Feat. Lil Wayne),[Kodak Black],Santa Baby,[Kylie Minogue]
9,Englishman In New York (Feat. Yu-ri (Blu-Swing)),[Re:Plus],It's The Most Wonderful Time Of The Year,[Andy Williams]


> 팝송끼리 많이 모여있었나?? 

In [123]:
#must have love - sg워너비, 브아걸
get_15_songs(112399)
# 아이템기반 협업필터링 / word2vec 사용 순위

,song_name,artist_name_basket,song_name,artist_name_basket
0,미리 메리 크리스마스 (Feat. 천둥 Of MBLAQ),[아이유],로맨틱 겨울 (Alpen Gondola Mix) (Feat. 김진호 Of SG워너비),[김진표]
1,크리스마스니까,"[성시경, 박효신, 이석훈, 서인국, VIXX (빅스)]",엉뚱한 상상 (White Christmas),[SUPER JUNIOR (슈퍼주니어)]
2,Happy Together,"[박지헌, 강민경 (다비치)]",Santa Claus Is Comin` To Town,[Mariah Carey]
3,하얀 설레임 (White Love),"[케이윌, 소유 (SOYOU), 정민]",스키장 가는 길,[LPG]
4,Christmas Time,"[성시경, 박효신, 서인국, 브라이언, 리사, 박학기, 김형중, 견우]",하루만 (White Remix),[조민혜]
5,겨울 고백,"[성시경, 박효신, 서인국, VIXX (빅스), 여동생]",러브송,[7공주]
6,하얀 겨울,"[김범수, 박정현]",메리 크리스마스 (With 화영),[언터쳐블]
7,All I Want For Christmas Is You,[Mariah Carey],우리 사랑하게 됐어요.,"[가인, 조권]"
8,로맨틱 겨울 (Feat. 김진호 Of SG워너비),[김진표],We Wish You A Merry Christmas & Feliz Navidad,[천상지희 더 그레이스 (The Grace)]
9,하얀 고백 (Lately),[인피니트],크리스마스의 기적2 (Narr. 홍록기),"[최아인, JQ, 공소원]"


> 겨울, 크리스마스 관련 노래들이 나온다.

In [85]:
# 암욜맨 - ss501
get_15_songs(400380)
# 아이템기반 협업필터링 / word2vec 사용 순위

,song_name,artist_name_basket,song_name,artist_name_basket
0,U R Man,[SS501],"쏘리 쏘리 (Sorry, Sorry)",[SUPER JUNIOR (슈퍼주니어)]
1,"쏘리 쏘리 (Sorry, Sorry)",[SUPER JUNIOR (슈퍼주니어)],주문-MIROTIC (Original Ver.),[동방신기 (TVXQ!)]
2,Ring Ding Dong,[SHINee (샤이니)],Diva,[애프터스쿨]
3,Love Like This (네게로),[SS501],니가 밉다,[2PM]
4,Wrong Number,[동방신기 (TVXQ!)],Love Like This (네게로),[SS501]
5,One Day,[엠투엠 (M To M)],토요일밤에,[손담비]
6,주문-MIROTIC (Original Ver.),[동방신기 (TVXQ!)],AH,[애프터스쿨]
7,몰랐었다 (Narration Mix Ver.) (Narr.김명민),[홍경민],Gee,[소녀시대 (GIRLS` GENERATION)]
8,마음 꽃,"[모세, 김주택]",만만하니,[유키스]
9,지울꺼야,[이수영],Again＆Again,[2PM]


> 수능 금지곡들

In [88]:
# 벛꽃 엔딩 - 버스커버스커
get_15_songs(132994)
# 아이템기반 협업필터링 / word2vec 사용 순위

,song_name,artist_name_basket,song_name,artist_name_basket
0,벚꽃 엔딩,[버스커 버스커],꽃송이가,[버스커 버스커]
1,꽃송이가,[버스커 버스커],첫사랑,[버스커 버스커]
2,봄봄봄,[로이킴],Love Blossom (러브블러썸),[케이윌]
3,봄 사랑 벚꽃 말고,"[HIGH4 (하이포), 아이유]",봄봄봄,[로이킴]
4,봄이 좋냐??,[10CM],연애시대 (Feat. Ra.D) (Narr. 한효주),[이승기]
5,Love Blossom (러브블러썸),[케이윌],정말로 사랑한다면,[버스커 버스커]
6,우연히 봄,"[로꼬, 유주 (여자친구)]",벚꽃보러갈까,[그릿]
7,봄인가 봐 (Spring Love),"[에릭남 (Eric Nam), 웬디 (WENDY)]",어느 봄날,[2AM]
8,봄 타나봐 (BOMTANABA),[서인국],꽃피는 봄이 오면,[김범수]
9,첫사랑,[버스커 버스커],개나리엔딩,[Bro]


> 양쪽 다 비슷하게 봄 관련 노래가 왔다.

In [96]:
# 이름이 뭐예요? - 포미닛
get_15_songs(638927)
# 아이템기반 협업필터링 / word2vec 사용 순위

,song_name,artist_name_basket,song_name,artist_name_basket
0,이름이 뭐예요?,[4minute],기대해,[걸스데이]
1,기대해,[걸스데이],Give It To Me,[씨스타]
2,Give It To Me,[씨스타],여자 대통령,[걸스데이]
3,NoNoNo,[Apink (에이핑크)],첫 사랑니 (Rum Pum Pum Pum),[f(x)]
4,여자 대통령,[걸스데이],예뻐 예뻐,[레이디스 코드]
5,Lovey-Dovey,[티아라],NoNoNo,[Apink (에이핑크)]
6,I Got A Boy,[소녀시대 (GIRLS` GENERATION)],와일드 (Wild),[나인뮤지스]
7,Roly-Poly,[티아라],I Got A Boy,[소녀시대 (GIRLS` GENERATION)]
8,Hot Issue,[4minute],DO YOU LOVE ME,[2NE1]
9,So Cool (쏘쿨),[씨스타],빠빠빠,[크레용팝]


> 비슷한 시기의 여자 아이돌 노래가 나왔다

In [129]:

# 태연 만약에
get_15_songs(645489)
# 아이템기반 협업필터링 / w645489ord2vec 사용 순위

,song_name,artist_name_basket,song_name,artist_name_basket
0,들리나요...,[태연 (TAEYEON)],들리나요...,[태연 (TAEYEON)]
1,응급실,[Izi],애인 있어요,[이은미]
2,시간을 거슬러,[린],응급실,[Izi]
3,눈의 꽃,[박효신],잊지말아요,[백지영]
4,잊지 말아요,[백지영],사랑해요 (I Love You),[태연 (TAEYEON)]
5,애인 있어요,[이은미],내사람,[환희]
6,총맞은것처럼,[백지영],시간을 거슬러,[린]
7,그리고 하나,[태연 (TAEYEON)],낙인,[임재범]
8,나 혼자서,[티파니 (TIFFANY)],하루만,"[손호영, 데니안]"
9,내 머리가 나빠서,[SS501],가슴 아파도,[환희]
